## Setup Spark environment

In [581]:
%%local
import os
import json

username = os.environ['JUPYTERHUB_USER']

configuration = dict(
    name = "%s-final-schedule" % username,
    executorMemory = "4G",
    executorCores = 4,
    numExecutors = 10,
    driverMemory = "4G",
    conf = {
#         "spark.pyspark.python": "/opt/anaconda3/bin/python3", # Use python3
        "spark.jars.repositories": "https://repos.spark-packages.org",
        "spark.jars.packages": "graphframes:graphframes:0.7.0-spark2.3-s_2.11"
    }
)

# set the application name as "<your_gaspar_id>-homework3"
get_ipython().run_cell_magic('configure', line="-f", cell=json.dumps(configuration))

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
4939,application_1618324153128_4034,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
4881,application_1618324153128_3956,pyspark,idle,Link,Link,None,
4900,application_1618324153128_3986,pyspark,idle,Link,Link,None,
4908,application_1618324153128_3996,pyspark,idle,Link,Link,None,
4910,application_1618324153128_3999,pyspark,idle,Link,Link,None,
4912,application_1618324153128_4001,pyspark,idle,Link,Link,None,
4919,application_1618324153128_4011,pyspark,idle,Link,Link,None,
4923,application_1618324153128_4015,pyspark,idle,Link,Link,None,
4924,application_1618324153128_4016,pyspark,idle,Link,Link,None,
4928,application_1618324153128_4022,pyspark,busy,Link,Link,None,
4930,application_1618324153128_4024,pyspark,busy,Link,Link,None,


In [582]:
%%send_to_spark -i username -t str -n username

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Successfully passed 'username' as 'username' to Spark kernel

In [583]:
print('We are using Spark %s' % spark.version)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

We are using Spark 2.3.2.3.1.4.0-315

In [584]:
sc.addPyFile('graphframes_graphframes-0.7.0-spark2.3-s_2.11.jar')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [585]:
from graphframes import *

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Load data into spark dataframes

In [586]:
stops = spark.read.orc('/data/sbb/orc/geostops')
stop_times = spark.read.csv("/data/sbb/csv/timetable/stop_times/2019/05/07/stop_times.csv", header=True).drop('pickup_type', 'drop_off_type')
routes = spark.read.csv('/data/sbb/csv/timetable/routes/2019/05/07/routes.csv', header=True )
calendar = spark.read.csv('/data/sbb/csv/timetable/calendar/2019/05/07/calendar.csv', header=True).drop('start_date','end_date')
trips = spark.read.csv('/data/sbb/csv/timetable/trips/2019/05/07/trips.csv', header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Filter out stops out of the 15km radius from Zürich HB

In [587]:
import pyspark.sql.functions as F
from pyspark.sql.functions import acos, asin, cos, sin, lit, toRadians, sqrt

def haversine(theta):
    return (lit(1) - cos(theta)) / lit(2)

def haversine_dist(latitude_x, longitude_x, latitude_y, longitude_y):
    latitude_x, longitude_x, latitude_y, longitude_y = toRadians(latitude_x), toRadians(longitude_x),\
                                                       toRadians(latitude_y), toRadians(longitude_y)
    h = haversine(latitude_x - latitude_y) + cos(latitude_x) * cos(latitude_y) * haversine(longitude_x - longitude_y)
    earth_radius = 6371.0
    return acos(lit(1) - lit(2) * h) * earth_radius

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [588]:
# Leave only stops in 15 km radius
zurich_HB_lat, zurich_HB_lon = 47.378177, 8.540192
stops = stops.withColumn('distance_zurich_HB', haversine_dist(lit(zurich_HB_lat), lit(zurich_HB_lon), stops.stop_lat, stops.stop_lon))
stops = stops.filter(stops.distance_zurich_HB <= 15)
all_info = stops.join(stop_times.join(trips.join(calendar, on='service_id'), on='trip_id'), on='stop_id')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Compute edge lists

### Distances between stops and walking edges

In [589]:
stops_start = stops.withColumnRenamed('stop_id', 'start_vertex')\
                   .withColumnRenamed('stop_lat', 'stop_lat_start')\
                   .withColumnRenamed('stop_lon', 'stop_lon_start')\
              
stops_end = stops.withColumnRenamed('stop_id', 'end_vertex')\
                 .withColumnRenamed('stop_lat', 'stop_lat_end')\
                 .withColumnRenamed('stop_lon', 'stop_lon_end')\
              

all_distances = stops_start.crossJoin(stops_end).withColumn('distance', haversine_dist(F.col('stop_lat_start'), F.col('stop_lon_start'),
                                                                                       F.col('stop_lat_end'), F.col('stop_lon_end')))

#                .withColumn('trip_id', F.lit('-1'))\
#                .withColumn('start_time', F.lit(-1))\

walking_speed = 0.05
walking_edges = all_distances.filter((F.col('distance') <= 0.5) & (F.col('start_vertex') != F.col('end_vertex')))\
               .withColumn('duration', F.ceil(F.col('distance') / walking_speed))\
               .withColumn('route_id', F.lit('walking'))\
               .select('start_vertex', 'end_vertex', 'duration', 'route_id')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Public transport edge list

In [590]:
@F.udf
def hour(timestamp):
    return timestamp[:2]

# keep only reasonable hours
min_day_hour, max_day_hour = 8, 20
all_info = all_info.filter(hour(F.col('arrival_time')).cast('int').between(min_day_hour, max_day_hour))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [591]:
from pyspark.sql import Window

@F.udf
def minutes(timestamp):
    return int(timestamp[:2]) * 60 + int(timestamp[3:5])

# needs columns :{trip_id, stop_sequence, arrival_time, departure_time}
def get_edges(trip_info):
    
    window = Window.partitionBy('trip_id').orderBy(F.col('stop_sequence').cast('int'))

    edges = trip_info.withColumn('arrival_time_minutes', minutes(F.col('arrival_time')).cast('int'))
    edges = edges.withColumn('departure_time_minutes', minutes(F.col('departure_time')).cast('int'))
    
    edges = edges.withColumn("prev_departure_minutes", F.lag(F.col('departure_time_minutes')).over(window))
    edges = edges.withColumn("duration", F.col('arrival_time_minutes') - F.col('prev_departure_minutes'))
    
    edges = edges.withColumn("start_vertex", F.lag(F.col('stop_id')).over(window))
    edges = edges.withColumnRenamed("stop_id", "end_vertex")
    edges = edges.withColumnRenamed('prev_departure_minutes', 'start_time')
    
    edges = edges.filter("prev_departure_minutes is not null") # removes start of trip
    
    return edges.select('start_vertex', 'end_vertex', 'start_time', 'duration', 'route_id').drop_duplicates(['start_vertex','end_vertex', 'start_time', 'route_id'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Reachable edges from Zurich HB

In [ ]:
graph_edges = transport_edges.select('start_vertex', 'end_vertex')\
                             .union(walking_edges.select('start_vertex', 'end_vertex'))\
                             .distinct()

In [ ]:
def filter_unreachable(vertices, edges):
    v = vertices.withColumnRenamed('stop_id', 'id')
    e = edges.withColumnRenamed('start_vertex', 'src').withColumnRenamed('end_vertex', 'dst')
    
    g = GraphFrame(v, e)
    
    cc = g.connectedComponents(algorithm='graphx')
    
    zurich_component = cc.filter("id == '8503000'").select('component').collect()[0][0]
    
    return cc[cc.component == zurich_component].select('id').withColumnRenamed('id', 'stop_id')

In [ ]:
reachable_stops = filter_unreachable(stops.select('stop_id').distinct(), graph_edges)

In [ ]:
reachable_edges = graph_edges.join(reachable_stops, graph_edges.start_vertex == reachable_stops.stop_id)\
                             .select('start_vertex', 'end_vertex')\
                             .join(reachable_stops, graph_edges.end_vertex == reachable_stops.stop_id)\
                             .select('start_vertex', 'end_vertex')

In [ ]:
reachable_edges.show()

### Save Results

In [ ]:
reachable_stops.write.parquet('/user/%s/final/parquet/reachable_stops' % username)
reachable_edges.write.parquet('/user/%s/final/parquet/reachable_edges' % username)

In [ ]:
%%local
from hdfs3 import HDFileSystem
hdfs = HDFileSystem(user='ebouille') # impersonate ebouille to read the file
hdfs.ls('/user/%s/final/parquet' % username)

## Shortest Path

In [592]:
inward_walking_edges = walking_edges.groupby('end_vertex').agg({'start_vertex': 'collect_set'})\
                                                              .withColumnRenamed('collect_set(start_vertex)', 'start_vertices')\
                                                              .toPandas().set_index('end_vertex').to_dict()['start_vertices']
walking_edge_duration = walking_edges.toPandas()
walking_edge_duration['key'] = walking_edge_duration.apply(lambda x: (x['start_vertex'], x['end_vertex']), axis=1)
walking_edge_duration = walking_edge_duration.set_index('key').to_dict()['duration']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [593]:
edges = get_edges(all_info.filter(F.col('monday') == '1')).sort(F.col('start_time').desc())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [569]:
# edges = edges.union(walking_edges.crossJoin(edges.select('start_time').distinct()).select('start_vertex', 'end_vertex', 'start_time', 'duration', 'route_id')).orderBy(F.col('start_time').desc()).cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [601]:
# This function assumes a (spark) dataframe with columns 'start_vertex', 'end_vertex', 'start_time', 'duration', 'route_id' 
# edges is a dataframe that represents the edges of the graph, 
# edges should be sorted by order of descending starting time
# inward_walking_edges is a map route_id -> list(route_id), representing the nodes from which we can walk to a given node
# walking_edge_duration is a map (route_id, route_id) -> float, representing the duration of the walk
# end_time should be in the format 'HH:MM'
def latest_departure_paths(edges, end_stop, end_time, inward_walking_edges, walking_edge_duration):
    
    # This function computes whether edge (u, v, t, dur, route_id) can be taken
    # returns update to node u if possible, and None otherwise
    def edge_valid(u, v, time, dur, route_id):
        
        cur_best = next_transfer.get(u)
        next_edge = next_transfer.get(v)
        
        if next_edge: # needs to be defined
            time_v, next_v, route_id_v, dur_v = next_edge
            
            if route_id == 'walking':
                if  route_id_v == 'walking' and dur_v + dur <= max_walking_time:
                    time = time_v - dur - transfer_time
                    if (not cur_best) or time > cur_best[0]:
                        return (time, next_v, route_id, dur + dur_v)
                elif route_id_v != 'walking':
                    time = time_v - dur - transfer_time
                    if (not cur_best) or time > cur_best[0]:
                        return (time, v, route_id, dur)
                      
            
            elif (route_id == route_id_v or v == end_stop) and time + duration <= time_v: # same route or last_stop
                if (not cur_best) or time > cur_best[0]: # can we improve
                    return (time, next_v, route_id, dur + dur_v)
            
            
            elif time + duration + transfer_time <= time_v: # transfer at node_v
                if (not cur_best) or time > cur_best[0]:
                    return (time, v, route_id, dur) 
        
        # In any other case no update can be made
        return None
                
    def time_to_minutes(timestamp):
        return int(timestamp[:2]) * 60 + int(timestamp[3:5])
    
    transfer_time = 2 # at least 2 minutes to transfer
    max_walking_time = 10

    next_transfer = {} # stores for each node u the node v 
                       # where the next transfer happens format (departure_time, v, route_id to v, duration)
    end_time = time_to_minutes(end_time)
    start_time = end_time - 120 # look only at edges departing at most 2 hours before end_time
    next_transfer[end_stop] = (end_time, end_stop, None, 0)
    
    edges = edges.filter(F.col('start_time').between(start_time, end_time)).toPandas()

    for _, row in edges.iterrows():
        
        start_vertex, end_vertex, start, duration, route_id = row.start_vertex, row.end_vertex, row.start_time, row.duration, row.route_id
        
        update = edge_valid(start_vertex, end_vertex, start, duration, route_id)
        if update:
            next_transfer[start_vertex] = update
                    
            w_start_vertex = start_vertex
            for w_end_vertex in inward_walking_edges.get(w_start_vertex, []):
                duration = walking_edge_duration[(w_start_vertex, w_end_vertex)]
                w_start = start 
                update = edge_valid(w_start_vertex, w_end_vertex, w_start, duration, 'walking')
                if update:
                     next_transfer[w_start_vertex] = update
                    
        
    return next_transfer

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [602]:
import time
start = time.time()
next_transfer = latest_departure_paths(edges, '8591221', '20:00', inward_walking_edges, walking_edge_duration)
end = time.time() 
print(end - start)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

50.3168818951

In [597]:
route_id_to_route_name = routes.select('route_id', 'route_desc', 'route_short_name').toPandas()
route_id_to_route_name['route_name'] = route_id_to_route_name['route_desc'] + ' ' + route_id_to_route_name['route_short_name']
route_id_to_route_name = route_id_to_route_name.set_index('route_id').to_dict()['route_name']
route_id_to_route_name['walking'] = 'walk'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [598]:
stop_id_to_stop = stops.select('stop_id', 'stop_name').toPandas().set_index('stop_id').to_dict()['stop_name']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [599]:
def minutes_to_timestamp(minutes):
    return str(minutes // 60) + ':' + str(minutes % 60).zfill(2)
    
def reconstruct_route(next_transfer, start_stop):
    time, next_stop, route_id, duration = next_transfer[start_stop]
    journey = ''
    while next_stop != start_stop:
        journey += """  \n
                        %s : %s 
                        | 
                        | %s (%s min) 
                        | 
                        %s : %s """ % (minutes_to_timestamp(time),\
                                       stop_id_to_stop[start_stop],\
                                       route_id_to_route_name[route_id],\
                                       duration, minutes_to_timestamp(time + duration), stop_id_to_stop[next_stop])
        start_stop = next_stop
        time, next_stop, route_id, duration = next_transfer[next_stop]
        
    return journey

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [625]:
print(reconstruct_route(next_transfer, '8590883')) # almost same as sbb

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  

                        18:37 : Wallisellen, Bützacker 
                        | 
                        | walk (5 min) 
                        | 
                        18:42 : Wallisellen, Lindenplatz   

                        18:44 : Wallisellen, Lindenplatz 
                        | 
                        | Bus 772 (2 min) 
                        | 
                        18:46 : Wallisellen, Blumenweg   

                        18:50 : Wallisellen, Blumenweg 
                        | 
                        | walk (6 min) 
                        | 
                        18:56 : Wallisellen, Ifang   

                        18:58 : Wallisellen, Ifang 
                        | 
                        | Bus 787 (3 min) 
                        | 
                        19:01 : Wallisellen, Glatt (Bus)   

                        19:07 : Wallisellen, Glatt (Bus) 
                        | 
                        | Bus 94 (1 min) 
                        | 
  

In [605]:
print(reconstruct_route(next_transfer, '8591353')) # almost same as sbb

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  

                        19:25 : Zürich, Schwert 
                        | 
                        | Tram 13 (6 min) 
                        | 
                        19:31 : Zürich, Escher-Wyss-Platz   

                        19:33 : Zürich, Escher-Wyss-Platz 
                        | 
                        | Tram 6 (10 min) 
                        | 
                        19:43 : Zürich, Central   

                        19:45 : Zürich, Central 
                        | 
                        | Bus 31 (11 min) 
                        | 
                        19:56 : Zürich, Kapfstrasse

In [610]:
print(reconstruct_route(next_transfer, '8590788')) # not in sbb

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  

                        19:13 : Schlieren, Brunngasse 
                        | 
                        | walk (2 min) 
                        | 
                        19:15 : Schlieren, Zentrum   

                        19:17 : Schlieren, Zentrum 
                        | 
                        | Bus 31 (39 min) 
                        | 
                        19:56 : Zürich, Kapfstrasse

In [613]:
print(reconstruct_route(next_transfer, '8591368')) # Correct

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  

                        19:38 : Zürich, Sihlquai/HB 
                        | 
                        | Tram 6 (5 min) 
                        | 
                        19:43 : Zürich, Central   

                        19:45 : Zürich, Central 
                        | 
                        | Bus 31 (11 min) 
                        | 
                        19:56 : Zürich, Kapfstrasse

In [624]:
print(reconstruct_route(next_transfer, next_transfer.keys()[100])) 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  

                        18:23 : Bremgarten AG, Zelgli 
                        | 
                        | walk (10 min) 
                        | 
                        18:33 : Bremgarten AG, Bahnhof   

                        18:35 : Bremgarten AG, Bahnhof 
                        | 
                        | Bus 444 (24 min) 
                        | 
                        18:59 : Zürich, Sihlcity   

                        19:23 : Zürich, Sihlcity 
                        | 
                        | walk (3 min) 
                        | 
                        19:26 : Zürich, Saalsporthalle   

                        19:28 : Zürich, Saalsporthalle 
                        | 
                        | Tram 5 (10 min) 
                        | 
                        19:38 : Zürich, Bürkliplatz   

                        19:41 : Zürich, Bürkliplatz 
                        | 
                        | Tram 8 (5 min) 
                        | 
                   